<a href="https://colab.research.google.com/github/ews46167-art/mgmt467-analytics-portfolio/blob/main/Lab1_AI_Assisted_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: AI‑Assisted SQL Foundations (Colab)**
**Course:** MGMT 467 — AI‑Assisted Big Data Analytics in the Cloud  
**When:** Unit 1 · Week 3 (Thursday)  
**Goal:** Use *Gemini as a co‑pilot* to write and understand basic SQL against the **Superstore** dataset in **BigQuery**.  
**You will practice:** Prompt engineering → SQL generation → Query execution → Interpretation.

> ✅ **Deliverable (submit link in Brightspace):** A completed `Lab1_AI_Assisted_SQL.ipynb` with prompts, code, outputs.


## ✅ What you need before starting
- A Google account with access to **Google Colab**.
- A **Google Cloud Project** with **BigQuery** API enabled.
- A dataset named `superstore_data` and a table named `sales` (your instructor or TA will provide details).
- (Optional) A GitHub repo to store your work for your team.


## 0) Connect to class GitHub repo
## Install and import libraries
## Class Repo: bigDataNCloud/mgmt_467_resources

In [ ]:
# Replace with your actual class repo URL
CLASS_REPO_URL = "https://github.com/YOUR-ORG-OR-USERNAME/class-repo.git"
CLASS_REPO_DIR = "/content/class-repo"

# If already cloned, skip re-downloading
import os
if not os.path.exists(CLASS_REPO_DIR):
    !git clone {CLASS_REPO_URL} {CLASS_REPO_DIR}
else:
    %cd {CLASS_REPO_DIR}
    !git pull

# Move into repo folder
%cd {CLASS_REPO_DIR}

print("✅ Class repository ready at", CLASS_REPO_DIR)

Cloning into '/content/class-repo'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: '/content/class-repo'
/content
✅ Class repository ready at /content/class-repo


In [ ]:
!pip -q install google-cloud-bigquery google-cloud-bigquery-storage db-dtypes pandas pyarrow kagglehub

In [ ]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import os, json, textwrap, pathlib, pprint
print("Libraries imported.")

Libraries imported.


## 1) Authenticate to GCP and set your project
Run the cell below. When prompted, authorize Colab to access your Google Cloud resources.

In [ ]:
auth.authenticate_user()
print("✅ Authenticated.")

# Set your GCP project (EDIT THIS!)
PROJECT_ID = "manifest-chain-471119-t8" # REPLACE THIS WITH YOUR ACTUAL GCP PROJECT ID
# assert PROJECT_ID != "manifest-chain-471119-t8", "Please set PROJECT_ID to your GCP project id." # Removed assertion
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# BigQuery client
bq = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client created for:", PROJECT_ID)

✅ Authenticated.
✅ BigQuery client created for: manifest-chain-471119-t8


### Quick connection test (optional)
This will try to read the first 5 rows from your Superstore table. If it fails, double‑check your dataset/table name.

In [ ]:
DATASET = "lab1_foundation"
TABLE   = "superstore"
BQ_TABLE = f"`{PROJECT_ID}.{DATASET}.{TABLE}`"

query = f"SELECT * FROM {BQ_TABLE} LIMIT 5"
try:
    df_preview = bq.query(query).to_dataframe()
    display(df_preview)
    print("✅ Connection OK.")
except Exception as e:
    print("⚠️ Could not query the table. Error below:")
    print(e)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,5769,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-LA-10001641,Office Supplies,Labels,Avery 518,3.15,1,0.0,1.5120
1,5770,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-PA-10002377,Office Supplies,Paper,Adams Telephone Message Book W/Dividers/Space ...,22.72,4,0.0,10.2240
2,9028,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10002597,Furniture,Furnishings,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,3,0.0,6.2244
3,9029,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10004864,Furniture,Furnishings,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,3,0.0,61.3824
4,8332,CA-2016-153269,2016-03-09,2016-03-12,First Class,PS-18760,Pamela Stobb,Consumer,United States,Andover,...,1810,East,OFF-ST-10004634,Office Supplies,Storage,"Personal Folder Holder, Ebony",11.21,1,0.0,3.3630


✅ Connection OK.


## 2) (Optional) Download a dataset with **KaggleHub**
Use **KaggleHub** to pull sample data locally into Colab (e.g., for offline exploration or to stage for GCS).
> This is **optional** for Lab 1 (we focus on BigQuery), but useful to practice data access workflows.
> https://www.kaggle.com/datasets/anandaramg/global-superstore

In [ ]:
import kagglehub, os, pathlib

DATASET_REF = "anandaramg/global-superstore"   # you can change this to any public dataset reference

download_path = kagglehub.dataset_download(DATASET_REF)
print("Path to dataset files:", download_path)

# List files
p = pathlib.Path(download_path)
print("Files:")
for f in p.glob("**/*"):
    if f.is_file():
        print("-", f)


Using Colab cache for faster access to the 'global-superstore' dataset.
Path to dataset files: /kaggle/input/global-superstore
Files:
- /kaggle/input/global-superstore/Global Superstore.txt


## 3) Prompting approach for Lab 1
You'll **paste prompts into Gemini** (in a separate tab), get back **SQL**, then paste that SQL here to execute.  
We’ll practice three core question types:
- **“What”** (lists & filters) → `SELECT`, `WHERE`, `DISTINCT`
- **“How many”** (counts by category) → `COUNT`, `GROUP BY`
- **“Who is best”** (rank/limit by metric) → `SUM`, `ORDER BY`, `LIMIT`

### 3A) “What” Question — `SELECT`, `WHERE`, `DISTINCT`
**Business Question:** *A manager wants a list of all **unique** product sub‑categories sold in the **West** region.*

**Paste this prompt into Gemini (no edits needed except the project id if you used a different table path):**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Find all the unique values in the 'Sub_Category' column, but only for rows where the 'Region' column is exactly 'West'.
```

**Then paste the SQL you get into the cell below** (replace the placeholder) and run it.

In [ ]:
sql_str = f"""
SELECT `Ship Mode`, COUNT(*) AS order_count
FROM {BQ_TABLE}
GROUP BY `Ship Mode`
ORDER BY order_count DESC
"""

df_what = bq.query(sql_str).to_dataframe()
print(f"Rows: {len(df_what)}")
display(df_what)

Rows: 4


,Ship Mode,order_count
0,Standard Class,5968
1,Second Class,1945
2,First Class,1538
3,Same Day,543


**Explain it back (metacognition):** In Gemini, paste:



> **Notes:** This SQL query counts the number of orders for each shipping mode in your Superstore data. It then presents these counts, ordered from the shipping mode with the most orders to the one with the fewest. This helps you see which shipping methods are used most frequently.

### 3B) “How many” Question — `COUNT`, `GROUP BY`
**Business Question:** *How many orders were placed in each **Ship Mode**?*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Count the total number of records for each unique value in the 'Ship_Mode' column.
# The final result should have two columns: 'Ship_Mode' and 'order_count'.
```


In [ ]:
sql_str = f"""
SELECT `Ship Mode`, COUNT(*) AS order_count
FROM {BQ_TABLE}
GROUP BY `Ship Mode`
ORDER BY order_count DESC
"""

df_howmany = bq.query(sql_str).to_dataframe()
display(df_howmany)

,Ship Mode,order_count
0,Standard Class,5968
1,Second Class,1945
2,First Class,1538
3,Same Day,543


### 3C) “Who is best” Question — `SUM`, `ORDER BY`, `LIMIT`
**Business Question:** *Identify the **top 5 most profitable customers**.*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`. The customer identifier is 'Customer_ID'.
# GOAL: Calculate the sum of 'Profit' for each customer. The final output should show the 'Customer_ID' and their total profit, sorted from highest to lowest profit, and limited to only the top 5 results.
```


In [ ]:
sql_str = f"""
SELECT `Customer ID`, SUM(Profit) AS total_profit
FROM {BQ_TABLE}
GROUP BY `Customer ID`
ORDER BY total_profit DESC
LIMIT 5
"""

df_best = bq.query(sql_str).to_dataframe()
display(df_best)

,Customer ID,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055


## 4) Challenge prompts (author your own)
Write **your own precise prompt** in a text cell (or comment) for each question below, then get SQL from Gemini and run it in the provided code cells.

**Challenge 1:** *What is the **average discount** for products in the **Technology** category sold in the **East** region?*  
**Challenge 2:** *How many **unique customers** has each **Segment** (Consumer/Corporate/Home Office) served?*

**Challenge 1:** # TASK: Generate a BigQuery SQL query. # CONTEXT: The table is `{PROJECT_ID}.lab1_foundation.superstore`. The discount is in the 'Discount' column. # GOAL: Calculate the average of the 'Discount' column. The calculation should only include rows where the 'Category' is 'Technology' AND the 'Region' is 'East'.

**Challenge 2** # TASK: Generate a BigQuery SQL query. # CONTEXT: The table is `{manifest-chain-471119-t8}.lab1_foundation.superstore`. The customer identifier is 'Customer ID'. # GOAL: Count the number of unique values in the 'Customer ID' column for each unique value in the 'Segment' column. The final output should have two columns: 'Segment' and 'unique_customers'.

In [ ]:
sql_str = """
SELECT AVG(Discount) AS avg_discount
FROM `manifest-chain-471119-t8.lab1_foundation.superstore`
WHERE Category = 'Technology' AND Region = 'East'
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)
df_ch1 = bq.query(sql_str).to_dataframe()
display(df_ch1)

,avg_discount
0,0.143364


In [ ]:
sql_str = """
SELECT Segment, COUNT(DISTINCT `Customer ID`) AS unique_customers
FROM `manifest-chain-471119-t8.lab1_foundation.superstore`
GROUP BY Segment
ORDER BY unique_customers DESC
"""
sql_str = sql_str.replace("YOUR_PROJECT_ID_HERE", PROJECT_ID)
df_ch2 = bq.query(sql_str).to_dataframe()
display(df_ch2)

,Segment,unique_customers
0,Consumer,409
1,Corporate,236
2,Home Office,148


## 5) Reflection (DIVE mindset)
- **Discover:** What did you find first?  
- **Investigate:** What alternate query or filter changed the story?  
- **Validate:** Where could the AI‑generated SQL be wrong or incomplete? How did you check?  
- **Extend:** Which stakeholder could use your results tomorrow? What action should they take?

- Discover: First I noticed how I am supposed to ask Gemini to do the work that I want it to. I think I was overthinking it to make it more complicated than it had to be. Based on the queries, I also found the distribution of ship modes, the most profitable customers, the average discount for technology products in the East region, and the number of unique customers per segment.
- Investigate: An alternate query could be to look at the total sales or profit per region or category, or to analyze the most popular products. For example, looking at sales per region might reveal different insights than just looking at profit or ship mode.

- Validate: The AI-generated SQL could be wrong if the prompt was ambiguous or if the AI misinterpreted the column names or the goal. I checked the queries by examining the results (e.g., checking if the counts made sense, if the top customers had high profit values) and comparing them to what I expected based on the business question. I also double-checked the column names used in the queries against the table schema (though we didn't explicitly display the schema here, it's good practice).
- Extend: A sales manager could use the results of the "Who is best" query to identify key customers for targeted marketing or loyalty programs. They could also use the "How many" query on ship modes to analyze logistics efficiency and potentially negotiate better rates for frequently used modes. The results from the challenge questions could inform regional sales strategies or customer segmentation efforts.

## 6) Save your work to GitHub (pick one of the options)
**Option A (recommended):** In Colab, go to **File → Save a copy in GitHub…** and select your team repo + folder (e.g., `labs/Unit1/`).  
**Option B (CLI, if you know git):**
```bash
# (In Colab) mount Drive, then clone/pull/push as usual with a PAT
# Be careful to NOT store secrets in the notebook.
```
Name the file **`Lab1_AI_Assisted_SQL.ipynb`** and push it to your team repo.